# Le TP des tours de Hanoi

## pour un affichage dans un notebook avec ipycanvas

Les tours seront modélisées par une liste de 3 piles.

Dans la première des deux implémentations données : une pile est une liste Python.

Vous devez manipuler une pile uniquement avec les fonctions ou méthodes fournies.

In [1]:
# Pour animation avec un canvas par appel de hanoi_animation()
from ipycanvas import Canvas, hold_canvas
from time import sleep

# Première implémentation pour les piles
def creer_pile():
    '''Renvoie une pile vide'''
    return []

def est_vide(pile):
    '''Renvoie un booléen, True si la pile est vide et False sinon'''
    return pile == []

def empiler(pile, element):
    '''Empile element au sommet de pile'''
    pile.append(element)
    
def depiler(pile):
    '''Renvoie et enlève la valeur du sommet de pile'''
    assert not est_vide(pile), "Pile vide"
    return pile.pop()

# Exercice 1 : valeur du sommet d'une pile
def sommet(pile):
    ''' Renvoie la valeur au sommet de la pile mais sans la supprimer de la pile '''
    if est_vide(pile):
        raise IndexError('pile vide')
    else:
        valeur = depiler(pile)
        empiler(pile, valeur)
    return valeur

# Exercice 2 : constitution d'une pile de n disques
def mettre_disques(pile, n):
    '''Met des disques de taille n à 1 sur la pile'''
    for i in range(n, 0, -1):
        empiler(pile, i)

# Exercice 3 : création de trois tours, une avec n disques    
def creation_tours(n):
    ''' Renvoie une liste de 3 piles,
    la première correspond à la pile des n disques,
    les autres étant vides.'''
    p0 = creer_pile()
    p1 = creer_pile()
    p2 = creer_pile()
    mettre_disques(p0, n)
    return [p0, p1, p2]


# Exercice 4 : déplacement d'un disque selon les règles
def deplacer(tours, origine, cible) :
    '''
    Déplace la valeur au sommet de la pile d’indice origine vers le sommet de la pile d’indice cible.
    Si le déplacement n’est pas possible, parce qu’il ne respecte pas les règles du jeu, les piles ne sont pas modifiées.
    '''
    if not est_vide(tours[origine]) and (est_vide(tours[cible]) or sommet(tours[cible]) > sommet(tours[origine])) :
        empiler(tours[cible], depiler(tours[origine]))
        
# Exercice 5 : résoudre de façon récursive
def resoudre(tours, n, origine, cible, interm):
    '''
    déplace les n premiers disques au sommet de la pile d’indice origine vers la pile d’indice cible,
    en utilisant éventuellement la pile d’indice interm comme pile intermédiaire pour les déplacements
    '''
    if n == 1:
        deplacer(tours, origine, cible)
    else:
        resoudre(tours, n-1, origine, interm, cible)
        deplacer(tours, origine, cible)
        resoudre(tours, n-1, interm, cible, origine)

# Exercice 6 : calcule le nombre de coups nécessaire pour résoubre
def nb_etapes(n):
    '''
    renvoie le nombre d’étapes nécessaires pour déplacer une pile de n disques, avec n > 1.
    '''
    if n == 1:
        return 1
    else:
        return 2*nb_etapes(n-1) + 1

# Exercice 7 : transforme la pile en tableau de taille n     
def pile_vers_tableau(pile, n):
    '''renvoie le tableau de taille n correspondant à pile.'''
    tab = [0] * n
    p2 = creer_pile()
    while not est_vide(pile):
        empiler(p2, depiler(pile))
    i = 0
    while not est_vide(p2):
        v = depiler(p2)
        tab[i] = v
        empiler(pile, v)
        i += 1
    return tab

# Exercice 8 : transforme les tours en matrice nx3
def tours_vers_matrice(tours, n):
    ''' renvoie le tableau à deux dimensions correspondant aux tours'''
    return [pile_vers_tableau(tours[i], n) for i in range(3)]



# Définition des caratéristiques pour un canvas unique :
pas = 24
couleur = {0:'black', 1:'red', 2:'blue', 3:'green', 4:'orange', 5:'purple', 6:'cyan', 7:'magenta', 8:'yellow', 9:'pink'}



# Affichage graphique dans un notebook jupyter avec ipycanvas
def affichage_animation(hanoi, matrice, n) :
    ''''''
        
    # Affichage d'un élément centrale d'une pile ou d'un disque selon les valeurs de la matrice et les couleurs du dictionnaire
    with hold_canvas(hanoi):
        hanoi.clear()
        for j in range(n-1, -1, -1): # parcours de droite à gauche des sous-tableaux
            for i in range(3): # parcours de haut en bas des tableaux
                k = matrice[i][j]
                hanoi.stroke_style = couleur[k]
                if k == 0:
                    hanoi.line_width = pas/2.5
                    hanoi.stroke_line(pas*(n+1)*(i+1), hanoi.height - (j+2) * pas, pas*(n + 1)*(i+1), hanoi.height - (j + 1) * pas)
                else:
                    hanoi.line_width = pas
                    hanoi.stroke_line(pas*((n+1)*(i+1)-k/2), hanoi.height-(j+1.5)*pas, pas*((n+1)*(i+1)+k/2), hanoi.height-(j+1.5)*pas)
    
        # Socle
        hanoi.stroke_style = couleur[0]
        hanoi.line_width = pas/2.5           
        hanoi.stroke_line(pas*(n+1)/2, hanoi.height-2*hanoi.line_width, pas*(n+1)*3.5, hanoi.height-2*hanoi.line_width)
    
    
def affichage_tours_animation(hanoi, tours, n) :
    affichage_animation(hanoi, tours_vers_matrice(tours, n), n)

    
def resoudre_animation(hanoi, tours, n_disques, n, origine, cible, interm):
    if n == 1:
        deplacer(tours, origine, cible)
        affichage_tours_animation(hanoi, tours, n_disques)
        sleep(1)
    else:
        resoudre_animation(hanoi, tours, n_disques, n-1, origine, interm, cible)
        deplacer(tours, origine, cible)
        affichage_tours_animation(hanoi, tours, n_disques)
        sleep(1)
        resoudre_animation(hanoi, tours, n_disques, n-1, interm, cible, origine)
        
        
def hanoi_animation(n) :
    '''affiche les étapes à suivre pour résoudre le problème des tours de Hanoi avec n disques'''
    print(f"Voici comment résoudre le problème des tours de Hanoi avec {n} disques en {nb_etapes(n)} déplacements :")
    hanoi = Canvas(width = pas*(1+n)*6, height = pas*(1+n))
    display(hanoi)
    affichage_animation(hanoi, tours_vers_matrice(creation_tours(n), n), n)
    sleep(3)    
    resoudre_animation(hanoi, creation_tours(n), n, n, 0, 2, 1) 
    
    
# Test    
hanoi_animation(2)

Voici comment résoudre le problème des tours de Hanoi avec 2 disques en 3 déplacements :


Canvas(height=72, width=432)

In [2]:
hanoi_animation(3)

Voici comment résoudre le problème des tours de Hanoi avec 3 disques en 7 déplacements :


Canvas(height=96, width=576)

In [3]:
hanoi_animation(5)

Voici comment résoudre le problème des tours de Hanoi avec 5 disques en 31 déplacements :


Canvas(height=144, width=864)

In [4]:
# Pour un affichage statique de chaque étape dans un nouveau canvas
from ipycanvas import Canvas, hold_canvas


# Première implémentation pour les piles
def creer_pile():
    '''Renvoie une pile vide'''
    return []

def est_vide(pile):
    '''Renvoie un booléen, True si la pile est vide et False sinon'''
    return pile == []

def empiler(pile, element):
    '''Empile element au sommet de pile'''
    pile.append(element)
    
def depiler(pile):
    '''Renvoie et enlève la valeur du sommet de pile'''
    assert not est_vide(pile), "Pile vide"
    return pile.pop()

# Exercice 1 : valeur du sommet d'une pile
def sommet(pile):
    ''' Renvoie la valeur au sommet de la pile mais sans la supprimer de la pile '''
    if est_vide(pile):
        raise IndexError('pile vide')
    else:
        valeur = depiler(pile)
        empiler(pile, valeur)
    return valeur

# Exercice 2 : constitution d'une pile de n disques
def mettre_disques(pile, n):
    '''Met des disques de taille n à 1 sur la pile'''
    for i in range(n, 0, -1):
        empiler(pile, i)

# Exercice 3 : création de trois tours, une avec n disques    
def creation_tours(n):
    ''' Renvoie une liste de 3 piles,
    la première correspond à la pile des n disques,
    les autres étant vides.'''
    p0 = creer_pile()
    p1 = creer_pile()
    p2 = creer_pile()
    mettre_disques(p0, n)
    return [p0, p1, p2]


# Exercice 4 : déplacement d'un disque selon les règles
def deplacer(tours, origine, cible) :
    '''
    Déplace la valeur au sommet de la pile d’indice origine vers le sommet de la pile d’indice cible.
    Si le déplacement n’est pas possible, parce qu’il ne respecte pas les règles du jeu, les piles ne sont pas modifiées.
    '''
    if not est_vide(tours[origine]) and (est_vide(tours[cible]) or sommet(tours[cible]) > sommet(tours[origine])) :
        empiler(tours[cible], depiler(tours[origine]))
        
# Exercice 5 : résoudre de façon récursive
def resoudre(tours, n, origine, cible, interm):
    '''
    déplace les n premiers disques au sommet de la pile d’indice origine vers la pile d’indice cible,
    en utilisant éventuellement la pile d’indice interm comme pile intermédiaire pour les déplacements
    '''
    if n == 1:
        deplacer(tours, origine, cible)
    else:
        resoudre(tours, n-1, origine, interm, cible)
        deplacer(tours, origine, cible)
        resoudre(tours, n-1, interm, cible, origine)

# Exercice 6 : calcule le nombre de coups nécessaire pour résoubre
def nb_etapes(n):
    '''
    renvoie le nombre d’étapes nécessaires pour déplacer une pile de n disques, avec n > 1.
    '''
    if n == 1:
        return 1
    else:
        return 2*nb_etapes(n-1) + 1

# Exercice 7 : transforme la pile en tableau de taille n     
def pile_vers_tableau(pile, n):
    '''renvoie le tableau de taille n correspondant à pile.'''
    tab = [0] * n
    p2 = creer_pile()
    while not est_vide(pile):
        empiler(p2, depiler(pile))
    i = 0
    while not est_vide(p2):
        v = depiler(p2)
        tab[i] = v
        empiler(pile, v)
        i += 1
    return tab

# Exercice 8 : transforme les tours en matrice nx3
def tours_vers_matrice(tours, n):
    ''' renvoie le tableau à deux dimensions correspondant aux tours'''
    return [pile_vers_tableau(tours[i], n) for i in range(3)]

# Affichage graphique dans un notebook jupyter avec ipycanvas
def affichage_canvas(matrice, n) :
    ''''''
    pas = 10
    hanoi = Canvas(width = pas*(1+n)*6, height = pas*(n+1))
    couleur = {0:'black', 1:'red', 2:'blue', 3:'green', 4:'orange', 5:'purple', 6:'cyan', 7:'magenta', 8:'yellow', 9:'pink'}
    display(hanoi)
    
    # Affichage d'un élément centrale d'une pile ou d'un disque selon les valeurs de la matrice et les couleurs du dictionnaire
    with hold_canvas(hanoi):
        for j in range(n-1, -1, -1): # parcours de droite à gauche des sous-tableaux
            for i in range(3): # parcours de haut en bas des tableaux
                k = matrice[i][j]
                hanoi.stroke_style = couleur[k]
                if k == 0:
                    hanoi.line_width = pas/2.5
                    hanoi.stroke_line(pas*(n+1)*(i+1), hanoi.height - (j+2) * pas, pas*(n + 1)*(i+1), hanoi.height - (j + 1) * pas)
                else:
                    hanoi.line_width = pas
                    hanoi.stroke_line(pas*((n+1)*(i+1)-k/2), hanoi.height-(j+1.5)*pas, pas*((n+1)*(i+1)+k/2), hanoi.height-(j+1.5)*pas)
    
        # Socle
        hanoi.stroke_style = couleur[0]
        hanoi.line_width = pas/2.5           
        hanoi.stroke_line(pas*(n+1)/2, hanoi.height-2*hanoi.line_width, pas*(n+1)*3.5, hanoi.height-2*hanoi.line_width)
    
    
def affichage_tours_canvas(tours, n) :
    affichage_canvas(tours_vers_matrice(tours, n), n)

    
def resoudre_canvas(tours, n_disques, n, origine, cible, interm):
    if n == 1:
        deplacer(tours, origine, cible)
        affichage_tours_canvas(tours, n_disques)
    else:
        resoudre_canvas(tours, n_disques, n-1, origine, interm, cible)
        deplacer(tours, origine, cible)
        affichage_tours_canvas(tours, n_disques)
        resoudre_canvas(tours, n_disques, n-1, interm, cible, origine)
        
def hanoi_etapes(n) :
    '''affiche les étapes à suivre pour résoudre le problème des tours de Hanoi avec n disques'''
    print(f"Pour résoudre le problème des tours de Hanoi avec {n} disques")
    affichage_canvas(tours_vers_matrice(creation_tours(n), n), n)
    print(f"Il vaut faut suivre les {nb_etapes(n)} étapes suivantes :")
    resoudre_canvas(creation_tours(n), n, n, 0, 2, 1)

# Test    
hanoi_etapes(2)

Pour résoudre le problème des tours de Hanoi avec 2 disques


Canvas(height=30, width=180)

Il vaut faut suivre les 3 étapes suivantes :


Canvas(height=30, width=180)

Canvas(height=30, width=180)

Canvas(height=30, width=180)

In [5]:
hanoi_etapes(3)

Pour résoudre le problème des tours de Hanoi avec 3 disques


Canvas(height=40, width=240)

Il vaut faut suivre les 7 étapes suivantes :


Canvas(height=40, width=240)

Canvas(height=40, width=240)

Canvas(height=40, width=240)

Canvas(height=40, width=240)

Canvas(height=40, width=240)

Canvas(height=40, width=240)

Canvas(height=40, width=240)

In [6]:
hanoi_etapes(4)

Pour résoudre le problème des tours de Hanoi avec 4 disques


Canvas(height=50, width=300)

Il vaut faut suivre les 15 étapes suivantes :


Canvas(height=50, width=300)

Canvas(height=50, width=300)

...

Canvas(height=50, width=300)

Canvas(height=50, width=300)

Canvas(height=50, width=300)

Canvas(height=50, width=300)

Canvas(height=50, width=300)

In [7]:
# Pour animation avec un canvas unique
from ipycanvas import Canvas, hold_canvas
from time import sleep

# Première implémentation pour les piles
def creer_pile():
    '''Renvoie une pile vide'''
    return []

def est_vide(pile):
    '''Renvoie un booléen, True si la pile est vide et False sinon'''
    return pile == []

def empiler(pile, element):
    '''Empile element au sommet de pile'''
    pile.append(element)
    
def depiler(pile):
    '''Renvoie et enlève la valeur du sommet de pile'''
    assert not est_vide(pile), "Pile vide"
    return pile.pop()

# Exercice 1 : valeur du sommet d'une pile
def sommet(pile):
    ''' Renvoie la valeur au sommet de la pile mais sans la supprimer de la pile '''
    if est_vide(pile):
        raise IndexError('pile vide')
    else:
        valeur = depiler(pile)
        empiler(pile, valeur)
    return valeur

# Exercice 2 : constitution d'une pile de n disques
def mettre_disques(pile, n):
    '''Met des disques de taille n à 1 sur la pile'''
    for i in range(n, 0, -1):
        empiler(pile, i)

# Exercice 3 : création de trois tours, une avec n disques    
def creation_tours(n):
    ''' Renvoie une liste de 3 piles,
    la première correspond à la pile des n disques,
    les autres étant vides.'''
    p0 = creer_pile()
    p1 = creer_pile()
    p2 = creer_pile()
    mettre_disques(p0, n)
    return [p0, p1, p2]


# Exercice 4 : déplacement d'un disque selon les règles
def deplacer(tours, origine, cible) :
    '''
    Déplace la valeur au sommet de la pile d’indice origine vers le sommet de la pile d’indice cible.
    Si le déplacement n’est pas possible, parce qu’il ne respecte pas les règles du jeu, les piles ne sont pas modifiées.
    '''
    if not est_vide(tours[origine]) and (est_vide(tours[cible]) or sommet(tours[cible]) > sommet(tours[origine])) :
        empiler(tours[cible], depiler(tours[origine]))
        
# Exercice 5 : résoudre de façon récursive
def resoudre(tours, n, origine, cible, interm):
    '''
    déplace les n premiers disques au sommet de la pile d’indice origine vers la pile d’indice cible,
    en utilisant éventuellement la pile d’indice interm comme pile intermédiaire pour les déplacements
    '''
    if n == 1:
        deplacer(tours, origine, cible)
    else:
        resoudre(tours, n-1, origine, interm, cible)
        deplacer(tours, origine, cible)
        resoudre(tours, n-1, interm, cible, origine)

# Exercice 6 : calcule le nombre de coups nécessaire pour résoubre
def nb_etapes(n):
    '''
    renvoie le nombre d’étapes nécessaires pour déplacer une pile de n disques, avec n > 1.
    '''
    if n == 1:
        return 1
    else:
        return 2*nb_etapes(n-1) + 1

# Exercice 7 : transforme la pile en tableau de taille n     
def pile_vers_tableau(pile, n):
    '''renvoie le tableau de taille n correspondant à pile.'''
    tab = [0] * n
    p2 = creer_pile()
    while not est_vide(pile):
        empiler(p2, depiler(pile))
    i = 0
    while not est_vide(p2):
        v = depiler(p2)
        tab[i] = v
        empiler(pile, v)
        i += 1
    return tab

# Exercice 8 : transforme les tours en matrice nx3
def tours_vers_matrice(tours, n):
    ''' renvoie le tableau à deux dimensions correspondant aux tours'''
    return [pile_vers_tableau(tours[i], n) for i in range(3)]



# Définition des caratéristiques pour un canvas unique :
pas = 24
hanoi = Canvas(width = 900, height = 50+pas)
couleur = {0:'black', 1:'red', 2:'blue', 3:'green', 4:'orange', 5:'purple', 6:'cyan', 7:'magenta', 8:'yellow', 9:'pink'}



# Affichage graphique dans un notebook jupyter avec ipycanvas
def affichage_animation(matrice, n) :
    ''''''
        
    # Affichage d'un élément centrale d'une pile ou d'un disque selon les valeurs de la matrice et les couleurs du dictionnaire
    with hold_canvas(hanoi):
        hanoi.clear()
        for j in range(n-1, -1, -1): # parcours de droite à gauche des sous-tableaux
            for i in range(3): # parcours de haut en bas des tableaux
                k = matrice[i][j]
                hanoi.stroke_style = couleur[k]
                if k == 0:
                    hanoi.line_width = pas/2.5
                    hanoi.stroke_line(pas*(n+1)*(i+1), hanoi.height - (j+2) * pas, pas*(n + 1)*(i+1), hanoi.height - (j + 1) * pas)
                else:
                    hanoi.line_width = pas
                    hanoi.stroke_line(pas*((n+1)*(i+1)-k/2), hanoi.height-(j+1.5)*pas, pas*((n+1)*(i+1)+k/2), hanoi.height-(j+1.5)*pas)
        # Titre
        hanoi.font = '32px courrier'
        hanoi.fill_text(f"Les tours de Hanoi avec {n} disques en {nb_etapes(n)} déplacements :", 10, 32)
        
        # Socle
        hanoi.stroke_style = couleur[0]
        hanoi.line_width = pas/2.5           
        hanoi.stroke_line(pas*(n+1)/2, hanoi.height-2*hanoi.line_width, pas*(n+1)*3.5, hanoi.height-2*hanoi.line_width)
    
    
def affichage_tours_animation(tours, n) :
    affichage_animation(tours_vers_matrice(tours, n), n)

    
def resoudre_animation(tours, n_disques, n, origine, cible, interm):
    if n == 1:
        deplacer(tours, origine, cible)
        affichage_tours_animation(tours, n_disques)
        sleep(0.5) 
    else:
        resoudre_animation(tours, n_disques, n-1, origine, interm, cible)
        deplacer(tours, origine, cible)
        affichage_tours_animation(tours, n_disques)
        sleep(0.5) 
        resoudre_animation(tours, n_disques, n-1, interm, cible, origine)
        
        
def hanoi_animation(n) :
    '''affiche les étapes à suivre pour résoudre le problème des tours de Hanoi avec n disques'''
    hanoi.height = 50 + pas*(1+n)
    display(hanoi)
    
    
    
    affichage_animation(tours_vers_matrice(creation_tours(n), n), n)
    sleep(1)    
    resoudre_animation(creation_tours(n), n, n, 0, 2, 1) 
    
    
# Test    
hanoi_animation(5)

Canvas(height=194, width=900)